# How to write multi-batch `BatchRequest` - `ConfiguredAsset` Example for Pandas

* A `BatchRequest` facilitates the return of one or more `batch(es)` of data from a configured `Datasource`. To find more about `Batches`, please refer to the [related documentation](https://docs.greatexpectations.io/docs/guides/connecting_to_your_data/how_to_get_one_or_more_batches_of_data_from_a_configured_datasource#1-construct-a-batchrequest). 
* A `BatchRequest` can return 0 or more Batches of data depending on the underlying data, and how it is configured. This guide will help you configure `BatchRequests` to return multiple batches, which can be used by
   1. Self-Initializing Expectations to estimate parameters
   2. DataAssistants to profile your data and create and Expectation suite self-intialized parameters.
   
* Note : Multi-batch BatchRequests are not supported in `RuntimeDataConnector`.

## FileSystem Example

### Example Directory

Imagine we have a directory of 12 csv files, each corresponding to 1 month of Taxi rider data

```
yellow_tripdata_sample_2020-01.csv
yellow_tripdata_sample_2020-02.csv
yellow_tripdata_sample_2020-03.csv
yellow_tripdata_sample_2020-04.csv
yellow_tripdata_sample_2020-05.csv
yellow_tripdata_sample_2020-06.csv
yellow_tripdata_sample_2020-07.csv
yellow_tripdata_sample_2020-08.csv
yellow_tripdata_sample_2020-09.csv
yellow_tripdata_sample_2020-10.csv
yellow_tripdata_sample_2020-11.csv
yellow_tripdata_sample_2020-12.csv
```


In [ ]:
import great_expectations as ge
from ruamel import yaml
from great_expectations.core.batch import BatchRequest

* Load `DataContext`

In [ ]:
data_context: ge.DataContext = ge.get_context()

### `ConfiguredDataConnector` Example

* Add `Datasource` named `taxi_multi_batch_configured_datasource` with two `ConfiguredAssetDataConnectors `. A key difference is in the `pattern` used to identify one Batch from another. 

* The first DataConnector is called `configured_data_connector_single_batch_asset`, which names two `assets`: `yellow_trip_data_jan` and `yellow_trip_data_feb`. 
    * This can be seen in the output of `test_yaml_config()`, which shows the 2 data assets, with 1 Batch each
    * Here is the output: 
    ```	
    Available data_asset_names (2 of 2):
		yellow_trip_data_feb (1 of 1): ['yellow_tripdata_sample_2020-02.csv']
		yellow_trip_data_jan (1 of 1): ['yellow_tripdata_sample_2020-01.csv']
    ```
    * **Note**: in this case we actually don't need a `group_name` defined if we are just saying our pattern was `yellow_tripdata_sample_2020-(01)\\.csv`. However, GE currently doesn't allow a `pattern` to be defined without `group_name` also defined. So in our case, we set regex defined in `pattern` to capture the `month`.

* A second DataConnector is called `configured_data_connector_multi_batch_asset`:
    * It defines one asset `yellow_tripdata_all_months`, that takes all 12 `yellow_tripdata_sample_2020` files and maps it to the same asset, with each of the 12 months corresponding to Batches for the asset. 
    * In order to do this, we define the `pattern` as `"yellow_tripdata_sample_(.*)\\.csv"` which matches all months, with one capture group `(.*)` defined as `month`. This allows the resulting Batches to be distinguishable from one another, and allows them be specified using a `batch_definition`.
    * The results can be seen in the output of `test_yaml_config()`, which shows 3 of the 12 Batches corresponding to `yellow_tripdata_all_months`
    * Here is the output:
 ```
 Available data_asset_names (1 of 1):
       yellow_tripdata_all_months (3 of 12): ['yellow_tripdata_sample_2020-01.csv', 'yellow_tripdata_sample_2020-02.csv', 'yellow_tripdata_sample_2020-03.csv']
 ```

In [ ]:
data_path: str = "../../../../test_sets/taxi_yellow_tripdata_samples/samples_2020"

datasource_config = {
    "name": "taxi_multi_batch_configured_datasource",
    "class_name": "Datasource",
    "module_name": "great_expectations.datasource",
    "execution_engine": {
        "module_name": "great_expectations.execution_engine",
        "class_name": "PandasExecutionEngine",
    },
    "data_connectors": {
        "configured_data_connector_single_batch_asset": {
            "class_name": "ConfiguredAssetFilesystemDataConnector",
            "base_directory": data_path,
            "assets": {
                "yellow_trip_data_jan":
                {
                    "group_names": ["month"],
                    "pattern": "yellow_tripdata_sample_2020-(01)\\.csv",
                },
                "yellow_trip_data_feb":
                {
                    "group_names": ["month"],
                    "pattern": "yellow_tripdata_sample_2020-(02)\\.csv",
                }
            },
        },
        
        "configured_data_connector_multi_batch_asset": {
            "class_name": "ConfiguredAssetFilesystemDataConnector",
            "base_directory": data_path,
            "assets": {
                "yellow_tripdata_all_months":{
                    "pattern": "yellow_tripdata_sample_(.*)\\.csv",
                    "group_names": ["month"],

                }
            },
        },
        
    },
}

data_context.test_yaml_config(yaml.dump(datasource_config))

In [ ]:
# add_datasource only if it doesn't already exist in our configuration
try:
    data_context.get_datasource(datasource_config["name"])
except ValueError:
    data_context.add_datasource(**datasource_config)

## BatchRequest

* Depending on which `DataConnector` you send a `BatchRequest` to, you will retrieve a different number of `Batches`

* Single Batch returned by `configured_data_connector_single_batch_asset` DataConnector.

In [ ]:
single_batch_batch_request: BatchRequest = BatchRequest(
    datasource_name="taxi_multi_batch_configured_datasource",
    data_connector_name="configured_data_connector_single_batch_asset",
    data_asset_name="yellow_trip_data_jan",
)

In [ ]:
batch_list = data_context.get_batch_list(batch_request=single_batch_batch_request)

In [ ]:
batch_list

* Multi Batch returned by the `configured_data_connector_multi_batch_asset` DataConnector.

In [ ]:
multi_batch_batch_request: BatchRequest = BatchRequest(
    datasource_name="taxi_multi_batch_configured_datasource",
    data_connector_name="configured_data_connector_multi_batch_asset",
    data_asset_name="yellow_tripdata_all_months",
)

In [ ]:
multi_batch_batch_list = data_context.get_batch_list(batch_request=multi_batch_batch_request)

In [ ]:
multi_batch_batch_list

* You can also get a single Batch from a multi-batch DataConnector by passing in `data_connector_query`. Index `-1` will return the most recent (month = `12`) batch. 

In [ ]:
single_batch_batch_request_from_multi: BatchRequest = BatchRequest(
    datasource_name="taxi_multi_batch_configured_datasource",
    data_connector_name="configured_data_connector_multi_batch_asset",
    data_asset_name="yellow_tripdata_all_months",
    data_connector_query={
        "index": -1 # month: 12
    }
)

In [ ]:
batch_list = data_context.get_batch_list(batch_request=single_batch_batch_request_from_multi)

In [ ]:
batch_list

In [ ]:
batch_list[0].to_dict() # 'batch_identifiers': {'month': '12'}},

# Using auto-initializing `Expectations` to generate parameters

* We will generate a `Validator` using our `multi_batch_batch_list`

In [ ]:
multi_batch_batch_list = data_context.get_batch_list(batch_request=multi_batch_batch_request)

In [ ]:
example_suite = data_context.create_expectation_suite(expectation_suite_name="example_configured_suite", overwrite_existing=True)

In [ ]:
validator = data_context.get_validator_using_batch_list(batch_list=multi_batch_batch_list, expectation_suite=example_suite)

* When you run methods on the validator, it will typically run on the most recent batch (index `-1`), even if the Validator has access to a longer Batch list. For example, notice that the `pickup_datetime` and `dropoff_datetime` below are all associated with December, indicating that it is with the most recent Batch.

In [ ]:
validator.head()

### Typical Workflow
* A `batch_list` becomes really useful when you are calculating parameters for auto-initializing Expectations, as they us a `RuleBasedProfiler` under-the-hood to calculate parameters.

* Here is an example running `expect_column_median_to_be_between()` by "guessing" at the `min_value` and `max_value`. 

In [ ]:
validator.expect_column_median_to_be_between(column="trip_distance", min_value=0, max_value=1)

* The observed value for the most recent batch (December/2020) is going to be `1.61`, which means the Expectation fails

* Now we run the same expectation again, but this time with `auto=True`. This means the `median` values are going to calculated across the `batch_list` associated with the `Validator` (ie 12 Batches for 2020), which gives the min value of `1.6` and the max value of `1.92`

In [ ]:
validator.expect_column_median_to_be_between(column="trip_distance", auto=True)

* The `auto=True` will also automatically run the Expectation against the most recent Batch (which has an observed value of `1.61`) and the Expectation will pass. 

* You can now save the `ExpectationSuite`.

In [ ]:
validator.save_expectation_suite()

### Running the `ExpectationSuite` against single `Batch`

Now the ExpectationSuite can be used to validate single batches using a Checkpoint. As before, we can use `data_connector_query` to specify the batch that we would like to run the `Checkpoint` on, but the recommended way would be to use the `batch_identifier` parameter, where we have set `month` to `01` to specify the January 2020 batch.

In [ ]:
multi_batch_batch_request: BatchRequest = BatchRequest(
    datasource_name="taxi_multi_batch_inferred_datasource",
    data_connector_name="inferred_data_connector_multi_batch_asset",
    data_asset_name="yellow_tripdata_sample_2020",
    #data_connector_query={
    #    "index": 0 # this one will correspond to Jan 2020
    #}
)


In [ ]:
checkpoint_config = {
    "name": "my_checkpoint",
    "config_version": 1,
    "class_name": "SimpleCheckpoint",
    "validations": [
        {
            "batch_request": single_batch_batch_request_from_multi,
            "expectation_suite_name": "example_configured_suite",
            "batch_identifiers":{"month": "01"} # batch_identifier month is set to 01
        }
    ],
}
data_context.add_checkpoint(**checkpoint_config)

In [ ]:
results = data_context.run_checkpoint(
    checkpoint_name="my_checkpoint"
)

In [ ]:
results.success